# **Entrenamiento**

## Preprocesamiento

Antes de entrenar los modelos de n-gramas, realizaremos un procedimiento para indicar que tan seguido las distintas oraciones empiezan o terminan con cierto token. 

La idea general es agregar el token \<s> al inicio de cada oración, y \</s> al final de estas. Para esto, se utiliza la función padded_everygram_pipeline, la cual recibe el orden de n-gramas mas alto, y el texto donde se va a iterar la función.


In [59]:
import os

def saveCorpus(corpus: list[list], path: str) -> None:
    """ Guarda un corpus generado al correr esta libreta en un archivo txt definido en path """
    with open(path, mode='a', encoding='utf-8') as f:
        for sentence in corpus:
            f.write(" ".join(word for word in sentence) + '\n')
    print("done.")

def getCorpus(path: str) -> list[list]:
    """ Recupera un corpus guardado en un archivo de txt """
    corpus = []
    with open(path, mode='r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            corpus.append(line.split())
    print("done.")
    return corpus

import random
random.seed(2023)
def splitCorpus(corpus: list[list], splitFactor=0.2) -> tuple:
    """" Separa los datos en un conjunto de entenamiento y uno de prueba, 
         de tal forma que si splitFactor = 0.2 entonces un 20% estará en 
         el conjunto de prueba y el restante el de entrenamiento. """
    splitFactor = max(0.2, splitFactor)
    N = len(corpus)

    randomCorpus = corpus.copy()
    random.shuffle(randomCorpus)
    trainCorpus = randomCorpus[int((N+1)*splitFactor):]
    testCorpus = randomCorpus[:int((N+1)*splitFactor)]

    return trainCorpus, testCorpus

datasetPath = os.path.join(os.getcwd(), "data")
data = getCorpus(datasetPath + "/cleanCorpus.txt")
X_train, X_test = splitCorpus(data)


done.


In [60]:
os.getcwd()
print(f"{len(X_train)} oraciones en el conjunto de entrenamiento.\n{len(X_test)} en el de pruebas.")

192716 oraciones en el conjunto de entrenamiento.
48179 en el de pruebas.


In [61]:
from nltk.lm.preprocessing import padded_everygram_pipeline

train1,vocab1=padded_everygram_pipeline(1,X_train)
train2,vocab2=padded_everygram_pipeline(2,X_train)
train3,vocab3=padded_everygram_pipeline(3,X_train)
train4,vocab4=padded_everygram_pipeline(3,X_train)

## Entrenamiento

Despues del procedimiento previo, procedemos a entrenar los modelos. Entrenaremos el Estimador de Máxima Verosimilitud (Maximum Likelihood Estimator o MLE), especificando el orden más alto de n-gramas a utilizar. Además, en el penúltimo modelo se aplicará suavizado de Laplace, y en el último el suavizado de Kneser-Ney.

La función MLE crea un vocabulario vacío, el cual se llena al ajustar el modelo con la función fit.


In [62]:
from nltk.lm import MLE
from nltk.lm import Laplace
from nltk.lm import AbsoluteDiscountingInterpolated

Lm1=MLE(1)
Lm2=MLE(2)
Lm3=Laplace(3)
Lm4=AbsoluteDiscountingInterpolated(3)

Lm1.fit(train1,vocab1)
Lm2.fit(train2,vocab2)
Lm3.fit(train3,vocab3)
Lm4.fit(train4,vocab4)

## Evaluando y utilizando modelos entrenados

Para evaluar a nuestros modelos, no utilizamos la probabilidad como una métrica, si no la perplejidad,
la cual es una cantidad que nos dice que tan bien una distribución de probabilidad o un modelo predice una muestra. Cuando un modelo de n-gramas predice bien a la muestra, la perplejidad estará mas cerca a 1.

En la evaluación del modelo se utiliza el conjunto de prueba test, utilizando la función lm.perplexity(test).



In [63]:
print("La perplejidad del modelo de unigramas es: " + str(Lm1.perplexity(X_test)))
print("La perplejidad del modelo de bigramas es: " + str(Lm2.perplexity(X_test)))
print("La perplejidad del modelo con suavizado de Laplace es: " + str(Lm3.perplexity(X_test)))
print("La perplejidad del modelo con suavizado de Laplace es: " + str(Lm4.perplexity(X_test)))

La perplejidad del modelo de unigramas es: inf
La perplejidad del modelo de bigramas es: inf
La perplejidad del modelo con suavizado de Laplace es: 39605.85358237703
La perplejidad del modelo con suavizado de Laplace es: inf


Generación de frases.

Se utiliza `lm.generate(Numero de palabras, random seed)`

In [170]:
print("Frase con modelo de unigramas:")

text=""
frase=Lm1.generate(15)
for word in frase:
    text=text+word+" "
print(text)

Frase con modelo de unigramas:
ciento las ya lo y un va de lo pero la propósito estuvimos Vida como 


In [171]:
print("Frase con modelo de bigramas:")

text=""
frase=Lm2.generate(15)
for word in frase:
    text=text+word+" "
print(text)

Frase con modelo de bigramas:
relacionado con crecimiento rescate </s> a conocer y vamos a estas ciudades del pasado ejerció 


In [165]:
print("Frase con modelo de suavizado de Laplace:")

text=""
frase=Lm3.generate(15)
for word in frase:
    text=text+word+" "
print(text)


Frase con modelo de suavizado de Laplace:
debería de existir algún oficio en donde perdían y en el informe sobre la Marina 


In [169]:
print("Frase con modelo de suavizado de librería:")

text=""
frase=Lm4.generate(15)
for word in Lm4.generate(15):
    text=text+word+" "
print(text)

Frase con modelo de suavizado de librería:
tercera dosis empezando con los maestros de Campeche del mar y se para el mundo 


## Conclusiones finales

En este trabajo abordamos la tarea de generar texto mediante un modelo de lenguaje basado en *n-gramas*, con el objetivo de producir oraciones en base a las mañaneras del presidente de México Lic. Andrés Manuel López Obrador.

La metodología fue:
1. Obtención de datos.
2. Preparación y análisis exploratorio de datos.
3. Elaboración del modelo.

### Obtención de datos

Para la obtención de los datos crudos utilizamos como fuente el repositorio de [@nostradata ](https://www.nostrodata.com) en [`GitHub`](https://github.com/NOSTRODATA/conferencias_matutinas_amlo) en la cual se encuentran transcripciones de múltiples mañaneras separadas por fecha y participante. Se utilizó una librería que nos permitia descargar contenido del repositorio mediante Python.

### Preparación y análisis exploratorio de datos

Una vez que se tuvo todo el corpus en un archivo de .txt se realizo una segmentación tanto por palabras como oraciones con el objetivo de realizar un análisis que nos permitiera tomar desiciones con respecto al procesamiento de los datos. Entre ellos se realizaron estadísticas simples sobre las palabras y oraciones segmentadas, como la distribución de tamaños y frecuencias, y gráficas para darse una idea del corpus.

Se tomó la desición de conservar únicamente carácteres alfanuméricos y de convertir en tokens `<UNK>` aquellas palabras poco frecuentes que aparecieran menos de 4 veces las desviación estandadar con respecto a la media.

Se realizon un separación de conjunto de entrenamiento del 80% de las oraciones y un 20% para el conjunto de pruebas.

### Elaboración del modelo

Estuvo bien padriuks